In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Portfolio General Settings
We assume the code is run at T before market open, so it all data on and before T-1 close. Therefore, the end_date should be today - BDay(1)

In [46]:
end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
vol_target = 0.5
max_leverage = 1

# Linear Optimization (Risk-Adjusted Return)

- How to determine the capital?

Difficulty here is portfolio might use different leverage per rebalance, looking at the asset MV might not give you the total portfolio capital (e.g. 70% leverage, your stocks only worth 70% of your capital now)

we can use the market value on Futu divides by the leverage used for last periods. This will give you the total captial for the portfolio

Then we use this capital for next rebalance

We shuold strategically fix this issue by reading capital from Futu or keep track the capital ourselves

In [73]:
portfolio = PortfolioLinearOpt(
    capital=186734.0206185567,    
    name='ETF Buy and Hold Portfolio',
    lookback_period=60,
    opt_freq=1,
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),    
    tc_model=TransactionCostFutu(),    
    systems=[
        TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),        
])

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize(gamma=10, hhi=0.2)
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-02-24 03:03:35,086 - TradingSubSystemSingle [META] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-02-23......
2024-02-24 03:03:35,099 - TradingSubSystemSingle [META] - INFO - Volatility Target = 50.0% | Price Volatility = 77.7% | Last Scale Factor = 0.64
2024-02-24 03:03:35,102 - TradingSubSystemSingle [TSLA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-02-23......
2024-02-24 03:03:35,111 - TradingSubSystemSingle [TSLA] - INFO - Volatility Target = 50.0% | Price Volatility = 39.9% | Last Scale Factor = 0.84
2024-02-24 03:03:35,119 - TradingSubSystemSingle [NVDA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-02-23......
2024-02-24 03:03:35,135 - TradingSubSystemSingle [NVDA] - INFO - Volatility Target = 50.0% | Price Volatility = 68.8% | Last Scale Factor = 0.73
2024-02-24 03:03:35,150 - TradingSubSystemSingle [AAPL] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-02-23

2024-02-24 03:03:35,241 - TradingSubSystemSingle [BRK-B] - INFO - Volatility Target = 50.0% | Price Volatility = 10.8% | Last Scale Factor = 4.52
2024-02-24 03:03:35,259 - TradingSubSystemSingle [SPY] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-02-23......
2024-02-24 03:03:35,291 - TradingSubSystemSingle [SPY] - INFO - Volatility Target = 50.0% | Price Volatility = 13.9% | Last Scale Factor = 3.58
2024-02-24 03:03:35,297 - TradingSubSystemSingle [QQQ] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-02-23......
2024-02-24 03:03:35,304 - TradingSubSystemSingle [QQQ] - INFO - Volatility Target = 50.0% | Price Volatility = 18.4% | Last Scale Factor = 2.71
100%|██████████| 560/560 [00:03<00:00, 176.94it/s]


,Rebalanced Portfolio,Optimized Portfolio,^SPX,Single - META (BAH1),Single - TSLA (BAH1),Single - NVDA (BAH1),Single - AAPL (BAH1),Single - EWY (BAH1),Single - DXJ (BAH2),Single - BRK-B (BAH2),Single - SPY (BAH1),Single - QQQ (BAH1)
Measure,,,,,,,,,,,,
Cumulative Return,1.613885,1.611471,1.062691,1.551173,0.586553,2.875664,1.042806,0.868359,1.769780,1.395853,1.105455,1.117611
Annualized Return,0.248612,0.247649,0.046620,0.336524,-0.126599,0.614998,0.060149,-0.035189,0.281018,0.172968,0.065184,0.084780
Annualized Volatility,0.221379,0.220127,0.190387,0.506046,0.494851,0.495268,0.284896,0.248864,0.164295,0.182946,0.190817,0.255621
Annualized Sharpe Ratio,0.964222,0.965334,0.060230,0.595543,-0.326868,1.170771,0.087738,-0.282650,1.496486,0.753315,0.157383,0.194147
Maximum Drawdown,-0.263014,-0.270507,-0.254251,-0.682479,-0.635372,-0.531787,-0.309294,-0.406293,-0.117846,-0.265790,-0.244964,-0.348280


# Equal Weighted Portfolios

In [74]:
# portfolio = PortfolioStandard(    
#     capital=200000,
#     name='ETF Buy and Hold Portfolio',    
#     rebalance_iter=RebalancerIter('0 0 * * Fri', 2),
#     tc_model=TransactionCostFutu(),    
#     systems=[
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),       
# ])

# portfolio.set_start_date(start_date)
# portfolio.set_end_date(end_date)
# portfolio.backtest_subsystems()
# portfolio.optimize()
# portfolio.backtest()
# portfolio.rebalance()
# portfolio.performance()

# Execute the Portfolio

In [76]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2024-02-24 03:04:28,850 - ExecutorFutu - INFO - market: US
2024-02-24 03:04:28,854 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2024-02-24 03:04:28,870 | 7736 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=32, host=127.0.0.1, port=11111, user_id=18214795
2024-02-24 03:04:29,014 | 7736 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=32


2024-02-24 03:04:32,019 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $186,734
2024-02-24 03:04:32,021 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - INFO - Generate trade position based on target capital of $186,734
2024-02-24 03:04:32,029 - ExecutorFutu - INFO - Execute Linear Optimized Portfolio (ETF Buy and Hold Portfolio) position based on 2024-02-23


2024-02-24 03:04:32,032 | 7736 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=33, host=127.0.0.1, port=11111, user_id=18214795


2024-02-24 03:04:32,043 - Futu - INFO - 7 Positions: US.SPY, US.QQQ, US.NVDA, US.META, US.EWY, US.DXJ, US.BRK.B


...

2024-02-24 03:04:35,839 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 508.6, 'qty': 9.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-02-24 03:04:35,840 | 7736 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=35
2024-02-24 03:04:38,848 | 7736 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=36, host=127.0.0.1, port=11111, user_id=18214795


2024-02-24 03:04:39,074 - Futu - INFO - Placed Order: {'code': 'US.QQQ', 'price': 438.02, 'qty': 12.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-02-24 03:04:39,075 | 7736 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=36


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.NVDA,英伟达,SELL,NORMAL,SUBMITTING,1321517799085448981,34.0,798.49,2024-02-23 14:04:32,2024-02-23 14:04:32,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-02-23
0,US.SPY,SPDR 标普500指数ETF,SELL,NORMAL,SUBMITTING,5608537048788534993,9.0,508.60,2024-02-23 14:04:35,2024-02-23 14:04:35,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-02-23
0,US.QQQ,纳指100ETF-Invesco QQQ Trust,SELL,NORMAL,SUBMITTING,3657712349890928439,12.0,438.02,2024-02-23 14:04:39,2024-02-23 14:04:39,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-02-23
